In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import os
import dlib
import cv2
import numpy as np
import csv
from google.colab import drive

# Mount Google Drive
drive.mount('/content/gdrive')

# Path to the filtered CSV file
combine_path = '/content/gdrive/MyDrive/PhD_Data/combined/Combined_dataset_utk_fgnet_morph_allface.csv'

# Read the filtered CSV file into a DataFrame
combine_df = pd.read_csv(combine_path)

# Define paths
image_folder = '/content/gdrive/MyDrive/PhD_Data/combined/Combined_dataset/db_final_5000_viola'
features_output_path = '/content/gdrive/MyDrive/PhD_Data/combined/Ratio_features/test_face_new_ratio_features.csv'

# Path to the dlib pre-trained shape predictor model
shape_predictor_path = '/content/gdrive/MyDrive/PhD_Data/combined/global_features/shape_predictor_68_face_landmarks.dat'

# Initialize dlib's face detector (HOG-based) and the shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(shape_predictor_path)

# Function to detect landmarks using dlib
def detect_landmarks(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    rects = detector(gray, 1)
    if len(rects) > 0:
        shape = predictor(gray, rects[0])
        landmarks = np.array([[shape.part(i).x, shape.part(i).y] for i in range(68)])
        return landmarks
    else:
        return None

# Function to calculate distances between points
def euclidean_distance(pt1, pt2):
    return np.linalg.norm(pt1 - pt2)

# Function to calculate anthropometric features
def calculate_anthropometric_features(landmarks):
    features = {}

    # Facial Width to Height Ratio
    face_width = euclidean_distance(landmarks[0], landmarks[16])
    face_height = euclidean_distance(landmarks[8], landmarks[27])
    features['Facial_Width_to_Height'] = face_width / face_height

    # Lower Face Height to Total Face Height Ratio
    lower_face_height = euclidean_distance(landmarks[33], landmarks[8])
    features['Lower_Face_Height_to_Total_Face_Height'] = lower_face_height / face_height

    # Inter-Eye Distance to Face Width
    inter_eye_distance = euclidean_distance(landmarks[39], landmarks[42])
    features['Inter_Eye_Distance_to_Face_Width'] = inter_eye_distance / face_width

    # Eyebrow Length to Inter-Eye Distance
    eyebrow_length_left = euclidean_distance(landmarks[17], landmarks[21])
    eyebrow_length_right = euclidean_distance(landmarks[22], landmarks[26])
    features['Eyebrow_Length_Left_to_Inter_Eye_Distance'] = eyebrow_length_left / inter_eye_distance
    features['Eyebrow_Length_Right_to_Inter_Eye_Distance'] = eyebrow_length_right / inter_eye_distance

    # Eye Height to Eye Width Ratio
    eye_height_left = euclidean_distance(landmarks[37], landmarks[41])
    eye_height_right = euclidean_distance(landmarks[43], landmarks[47])
    eye_width_left = euclidean_distance(landmarks[36], landmarks[39])
    eye_width_right = euclidean_distance(landmarks[42], landmarks[45])
    features['Eye_Height_Left_to_Eye_Width'] = eye_height_left / eye_width_left
    features['Eye_Height_Right_to_Eye_Width'] = eye_height_right / eye_width_right

    # Nose Length to Face Height Ratio
    nose_length = euclidean_distance(landmarks[27], landmarks[33])
    features['Nose_Length_to_Face_Height'] = nose_length / face_height

    # Nose Width to Inter-Eye Distance
    nose_width = euclidean_distance(landmarks[31], landmarks[35])
    features['Nose_Width_to_Inter_Eye_Distance'] = nose_width / inter_eye_distance

    # Mouth Width to Nose Width Ratio
    mouth_width = euclidean_distance(landmarks[48], landmarks[54])
    features['Mouth_Width_to_Nose_Width'] = mouth_width / nose_width

    # Mouth Height to Mouth Width Ratio
    mouth_height = euclidean_distance(landmarks[51], landmarks[57])
    features['Mouth_Height_to_Mouth_Width'] = mouth_height / mouth_width

    # Chin Height to Lower Face Height Ratio
    chin_height = euclidean_distance(landmarks[8], landmarks[57])
    features['Chin_Height_to_Lower_Face_Height'] = chin_height / lower_face_height

    # Jaw Width to Face Width Ratio
    jaw_width = euclidean_distance(landmarks[5], landmarks[11])
    features['Jaw_Width_to_Face_Width'] = jaw_width / face_width

    return features

# Debugging: Print the size of combine_df to ensure it's read correctly
print(f"Total images to process: {len(combine_df)}")

# Open the CSV file to write features
with open(features_output_path, 'a', newline='') as f:
    writer = csv.writer(f)
    # Write header if the file is empty
    if os.stat(features_output_path).st_size == 0:
        header = [
            'Facial_Width_to_Height',
            'Lower_Face_Height_to_Total_Face_Height',
            'Inter_Eye_Distance_to_Face_Width',
            'Eyebrow_Length_Left_to_Inter_Eye_Distance',
            'Eyebrow_Length_Right_to_Inter_Eye_Distance',
            'Eye_Height_Left_to_Eye_Width',
            'Eye_Height_Right_to_Eye_Width',
            'Nose_Length_to_Face_Height',
            'Nose_Width_to_Inter_Eye_Distance',
            'Mouth_Width_to_Nose_Width',
            'Mouth_Height_to_Mouth_Width',
            'Chin_Height_to_Lower_Face_Height',
            'Jaw_Width_to_Face_Width',
            'filename'
        ]
        writer.writerow(header)

    for idx, row in combine_df.iterrows():
        img_path = os.path.join(image_folder, row['filename'])

        # Load the image
        img = cv2.imread(img_path)

        if img is not None:
            landmark = detect_landmarks(img)

            if landmark is not None:
                print(f"Processing image: {row['filename']}")  # Debug print

                # Calculate anthropometric features and save to CSV
                features = calculate_anthropometric_features(landmark)
                features_list = list(features.values()) + [row['filename']]
                writer.writerow(features_list)

            else:
                print(f"No landmarks detected for: {row['filename']}")  # Debug print
        else:
            print(f"Image not loaded: {row['filename']}")  # Debug print


Streaming output truncated to the last 5000 lines.
Processing image: 314814_03F17.JPG
Processing image: 334098_01M16.JPG
Processing image: 340914_01M18.JPG
Processing image: 315314_03M19.JPG
Processing image: 274255_04M19.JPG
Processing image: 328956_02M16.JPG
Processing image: 319784_01M18.JPG
Processing image: 295812_01M18.JPG
Processing image: 327781_00F20.JPG
Processing image: 349689_01F20.JPG
Processing image: 315619_03M18.JPG
Processing image: 338453_03M17.JPG
Processing image: 332429_01M17.JPG
Processing image: 299759_01M19.JPG
Processing image: 297858_00M19.JPG
Processing image: 311147_01M17.JPG
Processing image: 257276_05M20.JPG
Processing image: 289084_00F19.JPG
Processing image: 338643_01M17.JPG
Processing image: 347897_01M19.JPG
Processing image: 331030_02M18.JPG
Processing image: 325647_01M16.JPG
Processing image: 325166_02M16.JPG
Processing image: 305601_00M17.JPG
Processing image: 313621_02M19.JPG
Processing image: 288075_07M19.JPG
Processing image: 325474_03M19.JPG
Proc

In [3]:
import pandas as pd

# Path to the features CSV file
features_output_path = '/content/gdrive/MyDrive/PhD_Data/combined/Ratio_features/test_face_new_ratio_features.csv'

# Read the CSV file into a DataFrame
features_ratio = pd.read_csv(features_output_path)

# Display the first few rows of the DataFrame
features_ratio

ParserError: Error tokenizing data. C error: Expected 13 fields in line 129, saw 14
